In [36]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100, cifar10
from keras.applications import VGG19
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.utils import to_categorical

# Load CIFAR-100 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Preprocess data
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels, 100)
test_labels = to_categorical(test_labels, 100)

# Load the VGG19 model pre-trained on ImageNet
vgg19_model = VGG19(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

vgg19_model.trainable = False

# Modify the model to include new top layers for CIFAR-100 classification
x = Flatten()(vgg19_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the full model
model = Model(inputs=vgg19_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, batch_size=128, epochs=20,
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/20
391/391 [==============================] - 11s 25ms/step - loss: 3.8196 - accuracy: 0.1361 - val_loss: 3.3596 - val_accuracy: 0.1985
Epoch 2/20
391/391 [==============================] - 9s 24ms/step - loss: 3.2090 - accuracy: 0.2232 - val_loss: 3.1176 - val_accuracy: 0.2345
Epoch 3/20
391/391 [==============================] - 9s 24ms/step - loss: 3.0307 - accuracy: 0.2558 - val_loss: 3.0058 - val_accuracy: 0.2575
Epoch 4/20
391/391 [==============================] - 9s 24ms/step - loss: 2.9304 - accuracy: 0.2758 - val_loss: 2.9292 - val_accuracy: 0.2755
Epoch 5/20
391/391 [==============================] - 9s 24ms/step - loss: 2.8572 - accuracy: 0.2900 - val_loss: 2.8994 - val_accuracy: 0.2826
Epoch 6/20
391/391 [==============================] - 9s 24ms/step - loss: 2.7990 - accuracy: 0.3003 - val_loss: 2.8571 - val_accuracy: 0.2898
Epoch 7/20
391/391 [==============================] - 9s 24ms/step - loss: 2.7538 - accuracy: 0.3091 - val_loss: 2.8213 - val_accuracy: 0.298

In [38]:
import pandas as pd
pd.DataFrame(history.history).to_csv('history.csv')